In [11]:
import math
import random
import sys
from collections import deque

import pygame

# ------------- Config -------------
WIDTH, HEIGHT = 900, 600
FPS = 60

PLAYER_RADIUS = 12
ECHO_RADIUS = 12
PLAYER_SPEED = 4.0

ECHO_DELAY_SEC = 4.0                     # how long before your echo begins
ECHO_START_AFTER_FRAMES = int(ECHO_DELAY_SEC)

SENTRY_COUNT = 4
SENTRY_SIZE = 18
SENTRY_BASE_SPEED = 1.8

SIGIL_RADIUS = 10
SNAG_RADIUS = 10                          # Time Snag power-up
SNAG_FREEZE_SEC = 2.25

SAFE_SPAWN_MARGIN = 50

BG_COLOR = (14, 16, 22)
LINE_COLOR = (40, 45, 60)
TEXT_COLOR = (230, 235, 255)
SUBTLE = (180, 190, 220)

PLAYER_COLOR = (100, 220, 255)
ECHO_COLOR = (255, 120, 180)
SIGIL_COLOR = (120, 255, 155)
SENTRY_COLOR = (255, 205, 90)
SNAG_COLOR = (170, 160, 255)

# ------------- Helpers -------------
def clamp(v, lo, hi):
    return max(lo, min(hi, v))

def dist2(a, b):
    dx, dy = a[0] - b[0], a[1] - b[1]
    return dx * dx + dy * dy

def rand_pos():
    return (
        random.randint(SAFE_SPAWN_MARGIN, WIDTH - SAFE_SPAWN_MARGIN),
        random.randint(SAFE_SPAWN_MARGIN, HEIGHT - SAFE_SPAWN_MARGIN),
    )

def far_from(point, min_d, tries=128):
    for _ in range(tries):
        p = rand_pos()
        if math.hypot(p[0] - point[0], p[1] - point[1]) >= min_d:
            return p
    return rand_pos()

# ------------- Entities -------------
class Player:
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)
        self.speed = PLAYER_SPEED

    @property
    def pos(self):
        return (self.x, self.y)

    def update(self, keys):
        vx = (keys[pygame.K_RIGHT] or keys[pygame.K_d]) - (keys[pygame.K_LEFT] or keys[pygame.K_a])
        vy = (keys[pygame.K_DOWN] or keys[pygame.K_s]) - (keys[pygame.K_UP] or keys[pygame.K_w])

        # normalize diagonal movement
        if vx or vy:
            length = math.hypot(vx, vy)
            vx /= length
            vy /= length

        self.x += vx * self.speed
        self.y += vy * self.speed

        self.x = clamp(self.x, PLAYER_RADIUS, WIDTH - PLAYER_RADIUS)
        self.y = clamp(self.y, PLAYER_RADIUS, HEIGHT - PLAYER_RADIUS)

    def draw(self, surf):
        pygame.draw.circle(surf, PLAYER_COLOR, (int(self.x), int(self.y)), PLAYER_RADIUS)
        pygame.draw.circle(surf, (255, 255, 255), (int(self.x), int(self.y)), 3)

class Echo:
    def __init__(self, path_deque):
        self.path = path_deque
        self.index = 0
        self.freeze_timer = 0.0
        self.active = False

    def reset(self):
        self.index = 0
        self.freeze_timer = 0.0
        self.active = False

    @property
    def pos(self):
        if not self.active or self.index >= len(self.path):
            return None
        return self.path[self.index]

    def update(self, dt):
        if not self.active:
            return
        if self.freeze_timer > 0:
            self.freeze_timer -= dt
            return
        if self.index < len(self.path) - 1:
            self.index += 1

    def draw(self, surf):
        p = self.pos
        if p is None:
            return
        color = ECHO_COLOR if self.freeze_timer <= 0 else (200, 200, 200)
        pygame.draw.circle(surf, color, (int(p[0]), int(p[1])), ECHO_RADIUS, 2)

class Sentry:
    def __init__(self, x, y, vx, vy, speed):
        self.x, self.y = float(x), float(y)
        self.vx, self.vy = vx, vy
        self.speed = speed

    @property
    def rect(self):
        return pygame.Rect(int(self.x - SENTRY_SIZE/2), int(self.y - SENTRY_SIZE/2), SENTRY_SIZE, SENTRY_SIZE)

    def update(self):
        self.x += self.vx * self.speed
        self.y += self.vy * self.speed

        bounce = False
        if self.x < SENTRY_SIZE/2 or self.x > WIDTH - SENTRY_SIZE/2:
            self.vx *= -1
            bounce = True
        if self.y < SENTRY_SIZE/2 or self.y > HEIGHT - SENTRY_SIZE/2:
            self.vy *= -1
            bounce = True

        if bounce:
            angle = math.atan2(self.vy, self.vx) + random.uniform(-0.2, 0.2)
            self.vx, self.vy = math.cos(angle), math.sin(angle)
            self.x = clamp(self.x, SENTRY_SIZE/2, WIDTH - SENTRY_SIZE/2)
            self.y = clamp(self.y, SENTRY_SIZE/2, HEIGHT - SENTRY_SIZE/2)

    def draw(self, surf):
        pygame.draw.rect(surf, SENTRY_COLOR, self.rect, border_radius=6)

# ------------- Game -------------
class Game:
    def __init__(self):
        pygame.init()
        pygame.display.set_caption("Echo Runner — dodge your own past")
        self.screen = pygame.display.set_mode((WIDTH, HEIGHT))
        self.clock = pygame.time.Clock()
        self.font = pygame.font.SysFont("arial", 24)
        self.big = pygame.font.SysFont("arial", 48, bold=True)
        self.mono = pygame.font.SysFont("consolas", 20)
        self.state = "menu"
        self.reset_world()

    def reset_world(self):
        self.player = Player(WIDTH // 2, HEIGHT // 2)
        self.path = deque(maxlen=ECHO_START_AFTER_FRAMES * 3)
        self.echo = Echo(self.path)
        self.frame = 0
        self.score = 0
        self.best = getattr(self, "best", 0)

        self.sentries = []
        for i in range(SENTRY_COUNT):
            x, y = rand_pos()
            ang = random.uniform(0, 2 * math.pi)
            vx, vy = math.cos(ang), math.sin(ang)
            speed = SENTRY_BASE_SPEED + random.uniform(-0.3, 0.6)
            self.sentries.append(Sentry(x, y, vx, vy, speed))

        self.sigil_pos = far_from(self.player.pos, 120)
        self.snag_pos = None
        self.snag_cooldown = random.randint(5 * FPS, 9 * FPS)

    def spawn_sigil(self):
        self.sigil_pos = far_from(self.player.pos, 150)

    def maybe_spawn_snag(self):
        if self.snag_pos is not None:
            return
        self.snag_cooldown -= 1
        if self.snag_cooldown <= 0:
            self.snag_pos = far_from(self.player.pos, 180)
            self.snag_cooldown = random.randint(8 * FPS, 14 * FPS)

    def draw_grid(self):
        self.screen.fill(BG_COLOR)
        step = 30
        for x in range(0, WIDTH, step):
            pygame.draw.line(self.screen, LINE_COLOR, (x, 0), (x, HEIGHT))
        for y in range(0, HEIGHT, step):
            pygame.draw.line(self.screen, LINE_COLOR, (0, y), (WIDTH, y))

    def draw_hud(self):
        txt = self.mono.render(f"Score: {self.score}", True, TEXT_COLOR)
        self.screen.blit(txt, (12, 10))
        if self.echo.active and self.echo.freeze_timer > 0:
            tleft = max(0.0, self.echo.freeze_timer)
            fr = self.mono.render(f"Echo frozen: {tleft:0.1f}s", True, SUBTLE)
            self.screen.blit(fr, (12, 34))

    def menu_screen(self):
        self.draw_grid()
        title = self.big.render("Echo Runner", True, TEXT_COLOR)
        sub = self.font.render("Avoid your own delayed clone.", True, SUBTLE)
        tip = self.font.render("Press SPACE to start • Move with WASD/Arrows • P to pause", True, SUBTLE)
        self.screen.blit(title, (WIDTH // 2 - title.get_width() // 2, 140))
        self.screen.blit(sub, (WIDTH // 2 - sub.get_width() // 2, 200))
        self.screen.blit(tip, (WIDTH // 2 - tip.get_width() // 2, 250))

    def pause_screen(self):
        shade = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA)
        shade.fill((0, 0, 0, 140))
        self.screen.blit(shade, (0, 0))
        t = self.big.render("Paused", True, TEXT_COLOR)
        self.screen.blit(t, (WIDTH // 2 - t.get_width() // 2, HEIGHT // 2 - 40))

    def gameover_screen(self):
        self.draw_grid()
        title = self.big.render("Game Over", True, TEXT_COLOR)
        s1 = self.font.render(f"Score: {self.score}   Best: {self.best}", True, SUBTLE)
        s2 = self.font.render("Press R to retry • ESC to quit", True, SUBTLE)
        self.screen.blit(title, (WIDTH // 2 - title.get_width() // 2, 160))
        self.screen.blit(s1, (WIDTH // 2 - s1.get_width() // 2, 220))
        self.screen.blit(s2, (WIDTH // 2 - s2.get_width() // 2, 260))

    def run(self):
        while True:
            dt = self.clock.tick(FPS) / 1000.0
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                if event.type == pygame.KEYDOWN:
                    if self.state == "menu" and event.key == pygame.K_SPACE:
                        self.state = "playing"
                    elif self.state == "playing":
                        if event.key == pygame.K_p:
                            self.state = "paused"
                        if event.key == pygame.K_ESCAPE:
                            pygame.quit()
                            sys.exit()
                    elif self.state == "paused" and event.key == pygame.K_p:
                        self.state = "playing"
                    elif self.state == "gameover":
                        if event.key == pygame.K_r:
                            self.reset_world()
                            self.state = "playing"
                        elif event.key == pygame.K_ESCAPE:
                            pygame.quit()
                            sys.exit()

            if self.state == "menu":
                self.menu_screen()
                pygame.display.flip()
                continue

            if self.state == "paused":
                self.draw_grid()
                self.draw_world(draw_hud=True)
                self.pause_screen()
                pygame.display.flip()
                continue

            if self.state == "gameover":
                self.gameover_screen()
                pygame.display.flip()
                continue

            keys = pygame.key.get_pressed()
            self.player.update(keys)

            self.path.append(self.player.pos)
            self.frame += 1

            if not self.echo.active and len(self.path) > ECHO_START_AFTER_FRAMES:
                self.echo.active = True
                self.echo.index = 0

            self.echo.update(dt)

            for s in self.sentries:
                s.update()

            for s in self.sentries:
                if s.rect.collidepoint(self.player.pos):
                    self.trigger_gameover()

            ep = self.echo.pos
            if ep is not None:
                if dist2(self.player.pos, ep) <= (PLAYER_RADIUS + ECHO_RADIUS) ** 2:
                    self.trigger_gameover()

            if self.sigil_pos and dist2(self.player.pos, self.sigil_pos) <= (PLAYER_RADIUS + SIGIL_RADIUS) ** 2:
                self.score += 1
                self.best = max(self.best, self.score)
                self.spawn_sigil()
                for s in self.sentries:
                    s.speed *= 1.03 + random.uniform(0.0, 0.02)

            self.maybe_spawn_snag()
            if self.snag_pos and dist2(self.player.pos, self.snag_pos) <= (PLAYER_RADIUS + SNAG_RADIUS) ** 2:
                self.echo.freeze_timer = SNAG_FREEZE_SEC
                self.snag_pos = None

            self.draw_grid()
            self.draw_world(draw_hud=True)
            pygame.display.flip()

    def draw_world(self, draw_hud=False):
        if self.sigil_pos:
            pygame.draw.circle(self.screen, SIGIL_COLOR, (int(self.sigil_pos[0]), int(self.sigil_pos[1])), SIGIL_RADIUS)
        if self.snag_pos:
            pygame.draw.circle(self.screen, SNAG_COLOR, (int(self.snag_pos[0]), int(self.snag_pos[1])), SNAG_RADIUS, 2)

        if len(self.path) > 5:
            step = max(1, len(self.path) // 200)
            for i in range(0, len(self.path), step):
                px, py = self.path[i]
                alpha = int(255 * (i / len(self.path)) * 0.25)
                surf = pygame.Surface((6, 6), pygame.SRCALPHA)
                pygame.draw.circle(surf, (120, 170, 220, alpha), (3, 3), 3)
                self.screen.blit(surf, (px - 3, py - 3))

        for s in self.sentries:
            s.draw(self.screen)

        self.echo.draw(self.screen)
        self.player.draw(self.screen)

        if draw_hud:
            self.draw_hud()

    def trigger_gameover(self):
        self.state = "gameover"

# ------------- Main -------------
if __name__ == "__main__":
    Game().run()


SystemExit: 

In [15]:
import pygame
import random
import sys

# Initialize pygame
pygame.init()

# Screen settings
WIDTH, HEIGHT = 600, 400
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Catch Avoid Game")

# Colors
WHITE = (255, 255, 255)
PLAYER_COLOR = (0, 200, 255)
BLOCK_COLOR = (200, 50, 50)

# Clock
clock = pygame.time.Clock()

# Player settings
PLAYER_WIDTH, PLAYER_HEIGHT = 60, 15
player_x = WIDTH // 2 - PLAYER_WIDTH // 2
player_y = HEIGHT - 40
PLAYER_SPEED = 7

# Block settings
BLOCK_WIDTH, BLOCK_HEIGHT = 40, 20
BLOCK_SPEED = 2  # ⚡ Half speed compared to before
blocks = []

# Font
font = pygame.font.SysFont("Arial", 26)

def draw_player(x, y):
    pygame.draw.rect(screen, PLAYER_COLOR, (x, y, PLAYER_WIDTH, PLAYER_HEIGHT), border_radius=4)

def draw_block(block):
    pygame.draw.rect(screen, BLOCK_COLOR, block, border_radius=4)

def game_over():
    text = font.render("GAME OVER", True, (255, 0, 0))
    screen.blit(text, (WIDTH // 2 - 80, HEIGHT // 2 - 20))
    pygame.display.flip()
    pygame.time.wait(2000)
    pygame.quit()
    sys.exit()

def main():
    global player_x, blocks

    running = True
    spawn_timer = 0

    while running:
        screen.fill(WHITE)

        # Events
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        # Key movement
        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT] and player_x > 0:
            player_x -= PLAYER_SPEED
        if keys[pygame.K_RIGHT] and player_x < WIDTH - PLAYER_WIDTH:
            player_x += PLAYER_SPEED

        # Spawn blocks
        spawn_timer += 1
        if spawn_timer > 40:  # every ~40 frames
            block_x = random.randint(0, WIDTH - BLOCK_WIDTH)
            block = pygame.Rect(block_x, 0, BLOCK_WIDTH, BLOCK_HEIGHT)
            blocks.append(block)
            spawn_timer = 0

        # Move blocks
        for block in blocks:
            block.y += BLOCK_SPEED
            draw_block(block)

            # Check collision
            player_rect = pygame.Rect(player_x, player_y, PLAYER_WIDTH, PLAYER_HEIGHT)
            if block.colliderect(player_rect):
                game_over()

        # Draw player
        draw_player(player_x, player_y)

        # Update display
        pygame.display.flip()
        clock.tick(60)

    pygame.quit()

if __name__ == "__main__":
    main()


SystemExit: 